In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
#!unrar x "/content/drive/MyDrive/train_opt.rar" "/content/"
#!unrar x "/content/drive/MyDrive/val_opt.rar" "/content/"

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import cv2

In [ ]:
from keras.applications import VGG16


In [ ]:
#df=pd.read_csv('df2.csv')


In [ ]:
#df['image']=df['image'].apply(lambda x: x if x[-4:]=='.png' else np.nan)

In [ ]:
#df=df.dropna().reset_index(drop=True)

In [ ]:
#df.dir='/'+df.type+'/'+df.image

In [ ]:
#df

In [ ]:
size=576

In [ ]:
train = ImageDataGenerator(rescale=1/255)
val = ImageDataGenerator(rescale=1/255)

In [ ]:
train_generator = train.flow_from_directory(
    directory=r"/content/train_opt/",
    target_size=(size, size),
    color_mode="rgb",
    batch_size=16,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

In [ ]:
#!pip install -U albumentations

In [ ]:
import albumentations as A
def aug(img):
    transform=A.compose[A.Blur(always_apply=False, p=0.05, blur_limit=(3, 5)),
                        ]
    transformed = transform(image=img)
    return transformed

In [ ]:


datagen = ImageDataGenerator(preprocessing_function= aug)

In [ ]:
valid_generator = train.flow_from_directory(
    directory=r"/content/val_opt/",
    target_size=(size, size),
    color_mode="rgb",
    batch_size=16,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/content/drive/MyDrive/cnn.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')



In [ ]:
model=tf.keras.applications.resnet50.ResNet50(include_top=True,
                                              input_shape=(size, size, 3),
                                              pooling=None,
                                              weights=None,#'imagenet',
                                              classes=102)
#model.add(layers.Dense(102))

In [ ]:
'''model = models.Sequential()
#model.add(conv_base)
model.add(layers.Conv2D(256, kernel_size=(3,3),activation='relu',input_shape=(size, size, 3)))
#model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(size, size, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
#model.add(layers.Dense(256, activation='relu'))
model.add(layers.Flatten())

model.add(layers.Dense(102))'''

"model = models.Sequential()\n#model.add(conv_base)\nmodel.add(layers.Conv2D(256, kernel_size=(3,3),activation='relu',input_shape=(size, size, 3)))\n#model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(size, size, 3)))\nmodel.add(layers.MaxPooling2D((2, 2)))\nmodel.add(layers.Conv2D(128, (3, 3), activation='relu'))\nmodel.add(layers.MaxPooling2D((2, 2)))\nmodel.add(layers.Conv2D(128, (3, 3), activation='relu'))\n#model.add(layers.Dense(256, activation='relu'))\nmodel.add(layers.Flatten())\n\nmodel.add(layers.Dense(102))"

In [ ]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 576, 576, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 582, 582, 3)  0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 288, 288, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator ,
                    epochs=40, 
                    validation_data=valid_generator,
                    callbacks=[earlyStopping, mcp_save, reduce_lr_loss])

Epoch 1/40
328/328 [==============================] - 409s 1s/step - loss: 3.7440 - accuracy: 0.1593 - val_loss: 3.4414 - val_accuracy: 0.1508 - lr: 0.0010
Epoch 2/40
328/328 [==============================] - 398s 1s/step - loss: 2.6882 - accuracy: 0.2753 - val_loss: 4.5826 - val_accuracy: 0.1503 - lr: 0.0010
Epoch 3/40
328/328 [==============================] - 399s 1s/step - loss: 2.2891 - accuracy: 0.3543 - val_loss: 16.3129 - val_accuracy: 0.0243 - lr: 0.0010
Epoch 4/40
328/328 [==============================] - 398s 1s/step - loss: 1.9498 - accuracy: 0.4394 - val_loss: 6.1288 - val_accuracy: 0.1540 - lr: 0.0010
Epoch 5/40
328/328 [==============================] - 397s 1s/step - loss: 1.7319 - accuracy: 0.4909 - val_loss: 4.6970 - val_accuracy: 0.3518 - lr: 0.0010
Epoch 6/40
328/328 [==============================] - 398s 1s/step - loss: 1.5129 - accuracy: 0.5557 - val_loss: 3.6052 - val_accuracy: 0.3170 - lr: 0.0010
Epoch 7/40
328/328 [==============================] - 398s 1s/s

In [ ]:
'''from keras import backend as K 

#model = load_model('model.h5')

inp = model.input                                         # input placeholder
out = [layer.output for layer in model.layers]            # all layer outputs
get_outputs = K.function([inp, K.learning_phase()], out)   

print(get_outputs)'''


In [ ]:
from keras import backend as K

layer_dict = dict([(layer.name, layer) for layer in model.layers])

# input tensor
input_tensor = model.input

# output tensor of the given layer
layer_output = layer_dict['conv2d_1'].output

# get the output with respect to the input
func = K.function([input_tensor], [layer_output])

In [ ]:
func

In [ ]:
img = cv2.imread('/content/crop2_DJI_0066_1.jpg')
#x = img_to_array(img)
#x = x.reshape((1,) + x.shape) 
#img /= 255
img=img/255
img=cv2.resize(img, (576, 576))

In [ ]:
img=img.reshape(1, 576, 576, 3)

In [ ]:
model=tf.keras.models.load_model('/content/drive/MyDrive/mdl_wts65.hdf5')

In [ ]:
model_ex = tf.keras.Model(inputs=model.inputs,
                       outputs=model.get_layer(name="avg_pool").output)

In [ ]:
x = tf.expand_dims(img, axis=0)
y = model_ex(x[0])

In [ ]:
from tqdm import tqdm

for i in tqdm(train_generator):
    x = tf.expand_dims(train_generator[0][0], axis=0)
    y = model_ex(x[0])
    arr.append(list(y[0]))

In [ ]:
'''layer_outs = func([img, 1.])                                                
print(layer_outs)'''


In [ ]:
#preds=model.predict(valid_generator)